### Converts hundreds of CSVs to a small number of Parquet files so they can be uploaded more efficiently to Snowflake and then combined

In [1]:
'''
pip install dask[dataframe] # install dask library in terminal for dataframe handling
pip install parquet-tools # Used to view content of parquet files in CLI
'''

'\npip install dask[dataframe] # install dask library in terminal for dataframe handling\npip install parquet-tools # Used to view content of parquet files in CLI\n'

In [2]:
import pandas as pd
import os
import glob
import dask.dataframe as dd
import gc

In [ ]:
folder_path = ### #change to your folder path

In [ ]:
#Count the number of files in folder pathway
total_files = 0
files_in_folder = os.listdir(folder_path)
csv_files = [file for file in files_in_folder if file.endswith('.csv')]
csv_file_paths = [os.path.join(folder_path, file) for file in csv_files]
df = dd.read_csv(csv_file_paths, dtype=str)
for file in csv_file_paths:
  total_files += 1
print(f"Number of files: {total_files}")
df.head()

In [ ]:
#List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

#List to store individual DataFrames
dask_dfs = []

#Loop over each CSV file
for file in csv_files:
  try:
    file_path = os.path.join(folder_path, file)

    #Read the CSV file into a Dask DataFrame
    df = dd.read_csv(file_path, dtype=str, on_bad_lines='skip', engine='python')

    print(os.path.basename(file))

    #Add a new column with the filename
    basename = os.path.basename(file)
    df['SNAPSHOT_DATE'] = pd.to_datetime(basename.replace('Contact Backup ','').replace('.csv', '')) # Replace file name to only have the date

    #Append the DataFrame to the list
    dask_dfs.append(df)
  except Exception as e:
    print(f"Error reading file: {file}, Error: {e}")
    continue

Contact Backup December 01 2024.csv
Contact Backup February 01 2025.csv
Contact Backup January 01 2025.csv
Contact Backup November 02 2024.csv
Contact Backup October 02 2024.csv


In [ ]:
#Concatenate all DataFrames into a single Dask DataFrame
dask_df = dd.concat(dask_dfs, axis=0)

#Display the first few rows to test if SNAPSHOT_DATE column was added
dask_df.head() 

TEST

In [ ]:
num_rows = dask_df.shape[0].compute() // 3 #Counts how many rows are in the dataframe

In [ ]:
print(num_rows) #Use this number to help determine the number of parquet files wanted, it's suggested that a single file shouldn't hold more than 2,000,000 rows

866542


In [ ]:
#CHANGE npartitions to number of parquet files you want as output
df_repartitioned = dask_df.repartition(npartitions=3)

In [ ]:
#Save the Dask DataFrame to parquet files
parquet_path = ##
df_repartitioned.to_parquet(parquet_path, write_index=False) #Change to output pathway

In [ ]:
parquet-tools show [file pathway] # Use command in terminal to view content of parquet file. Change [file pathway] to your file path